# Spotify - Tunnel Playlist
## What makes a good tunnel playlist?
The goal of this project is to looking into [Spotify's Tunnel playlist](https://open.spotify.com/playlist/0OwZcV6bAN2jqqXmA23Jnw?si=b01b353689a34be8) and identify the key track elements that makes for a good tunnel track.

## Helpful Resources:
- [Spotify API Dcoumentation](https://developer.spotify.com/documentation/web-api/reference/#/)
- [Spotipy - Python library](https://spotipy.readthedocs.io/en/2.19.0/)

In [15]:
import config
import pandas as pd
import json
import requests
import spotipy
from spotipy.oauth2 import SpotifyOAuth
import scipy.stats as stats
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
from functools import reduce
%matplotlib inline

## Get Authentication

In [2]:
redirect_uri = 'http://localhost:7777/callback'

sp = spotipy.Spotify(auth_manager=SpotifyOAuth(client_id=config.client_id,
                                               client_secret=config.client_secret,
                                               redirect_uri=redirect_uri,
                                               scope="user-read-recently-played"))

### Playlist Data

In [3]:
playlist_id = '0OwZcV6bAN2jqqXmA23Jnw?si=7e0fd30e5bd44bfa'

track_json = sp.playlist(playlist_id, fields=None, market=None, additional_types=('track',))

playlist_df = pd.json_normalize(track_json['tracks']['items'])

# replace all '.' with '_' in column headers
playlist_df.columns = playlist_df.columns.str.replace("[.]", "_")

playlist_df.head()

,added_at,is_local,primary_color,added_by_external_urls_spotify,added_by_href,added_by_id,added_by_type,added_by_uri,track_album_album_type,track_album_artists,...,track_id,track_is_local,track_name,track_popularity,track_preview_url,track_track,track_track_number,track_type,track_uri,video_thumbnail_url
0,2020-11-18T06:14:13Z,False,None,https://open.spotify.com/user/toricopeland_,https://api.spotify.com/v1/users/toricopeland_,toricopeland_,user,spotify:user:toricopeland_,single,[{'external_urls': {'spotify': 'https://open.s...,...,0I1eFRytp4XRhLCjT6tZm7,False,I Can't Handle Change,83,https://p.scdn.co/mp3-preview/4c2554c0416d42a4...,True,1,track,spotify:track:0I1eFRytp4XRhLCjT6tZm7,None
1,2020-11-18T06:13:55Z,False,None,https://open.spotify.com/user/toricopeland_,https://api.spotify.com/v1/users/toricopeland_,toricopeland_,user,spotify:user:toricopeland_,album,[{'external_urls': {'spotify': 'https://open.s...,...,0ntQJM78wzOLVeCUAW7Y45,False,Sex on Fire,81,https://p.scdn.co/mp3-preview/8cca3506fa42dd9c...,True,3,track,spotify:track:0ntQJM78wzOLVeCUAW7Y45,None
2,2020-11-18T06:31:22Z,False,None,https://open.spotify.com/user/toricopeland_,https://api.spotify.com/v1/users/toricopeland_,toricopeland_,user,spotify:user:toricopeland_,album,[{'external_urls': {'spotify': 'https://open.s...,...,6N22FZs2ZhPBYi3b9XPajV,False,Still Don't Know My Name,80,https://p.scdn.co/mp3-preview/2f3df1d334b61c34...,True,12,track,spotify:track:6N22FZs2ZhPBYi3b9XPajV,None
3,2020-11-18T06:19:50Z,False,None,https://open.spotify.com/user/toricopeland_,https://api.spotify.com/v1/users/toricopeland_,toricopeland_,user,spotify:user:toricopeland_,album,[{'external_urls': {'spotify': 'https://open.s...,...,1Fwj0wThn3kTg8D7KgWdsU,False,My Tears Are Becoming A Sea,62,https://p.scdn.co/mp3-preview/b320a486166d237d...,True,1,track,spotify:track:1Fwj0wThn3kTg8D7KgWdsU,None
4,2020-11-18T07:40:50Z,False,None,https://open.spotify.com/user/toricopeland_,https://api.spotify.com/v1/users/toricopeland_,toricopeland_,user,spotify:user:toricopeland_,album,[{'external_urls': {'spotify': 'https://open.s...,...,7Jh1bpe76CNTCgdgAdBw4Z,False,Heroes - 2017 Remaster,75,https://p.scdn.co/mp3-preview/92caa5df4a00974b...,True,3,track,spotify:track:7Jh1bpe76CNTCgdgAdBw4Z,None


### Get Artist IDs

In [11]:
artist_id_ls = []

for i in range(len(playlist_df)):
    artist_id_ls.append(track_json['tracks']['items'][i]['track']['album']['artists'][0]['id'])

# Add artist ids to playlist data frame 
playlist_df['artist_id'] = artist_id_ls

playlist_df.head()

,added_at,is_local,primary_color,added_by_external_urls_spotify,added_by_href,added_by_id,added_by_type,added_by_uri,track_album_album_type,track_album_artists,...,track_popularity,track_preview_url,track_track,track_track_number,track_type,track_uri,video_thumbnail_url,release_date_clean,release_date_year,artist_id
0,2020-11-18T06:14:13Z,False,None,https://open.spotify.com/user/toricopeland_,https://api.spotify.com/v1/users/toricopeland_,toricopeland_,user,spotify:user:toricopeland_,single,[{'external_urls': {'spotify': 'https://open.s...,...,83,https://p.scdn.co/mp3-preview/4c2554c0416d42a4...,True,1,track,spotify:track:0I1eFRytp4XRhLCjT6tZm7,None,2010-03-14,2010,7dIpKWlEeAljA20vFJ82RD
1,2020-11-18T06:13:55Z,False,None,https://open.spotify.com/user/toricopeland_,https://api.spotify.com/v1/users/toricopeland_,toricopeland_,user,spotify:user:toricopeland_,album,[{'external_urls': {'spotify': 'https://open.s...,...,81,https://p.scdn.co/mp3-preview/8cca3506fa42dd9c...,True,3,track,spotify:track:0ntQJM78wzOLVeCUAW7Y45,None,2008-09-23,2008,2qk9voo8llSGYcZ6xrBzKx
2,2020-11-18T06:31:22Z,False,None,https://open.spotify.com/user/toricopeland_,https://api.spotify.com/v1/users/toricopeland_,toricopeland_,user,spotify:user:toricopeland_,album,[{'external_urls': {'spotify': 'https://open.s...,...,80,https://p.scdn.co/mp3-preview/2f3df1d334b61c34...,True,12,track,spotify:track:6N22FZs2ZhPBYi3b9XPajV,None,2019-10-04,2019,2feDdbD5araYcm6JhFHHw7
3,2020-11-18T06:19:50Z,False,None,https://open.spotify.com/user/toricopeland_,https://api.spotify.com/v1/users/toricopeland_,toricopeland_,user,spotify:user:toricopeland_,album,[{'external_urls': {'spotify': 'https://open.s...,...,62,https://p.scdn.co/mp3-preview/b320a486166d237d...,True,1,track,spotify:track:1Fwj0wThn3kTg8D7KgWdsU,None,2011-01-01,2011,63MQldklfxkjYDoUE4Tppz
4,2020-11-18T07:40:50Z,False,None,https://open.spotify.com/user/toricopeland_,https://api.spotify.com/v1/users/toricopeland_,toricopeland_,user,spotify:user:toricopeland_,album,[{'external_urls': {'spotify': 'https://open.s...,...,75,https://p.scdn.co/mp3-preview/92caa5df4a00974b...,True,3,track,spotify:track:7Jh1bpe76CNTCgdgAdBw4Z,None,1977-01-01,1977,0oSGxfWSnnOXhD2fKuz2Gy


### Get Track Data from Playlist

In [5]:
tracks_data = sp.tracks(playlist_df['track_id'].tolist())
tracks_df = pd.json_normalize(tracks_data['tracks'])

# replace all '.' with '_' in column headers
tracks_df.columns = tracks_df.columns.str.replace("[.]", "_")

# change column header 'id' to 'track_id'
tracks_df.rename(columns={'id':'track_id'}, inplace=True)

tracks_df.head()

,artists,available_markets,disc_number,duration_ms,explicit,href,track_id,is_local,name,popularity,...,album_id,album_images,album_name,album_release_date,album_release_date_precision,album_total_tracks,album_type,album_uri,external_ids_isrc,external_urls_spotify
0,[{'external_urls': {'spotify': 'https://open.s...,"[AD, AE, AG, AL, AM, AO, AR, AT, AU, AZ, BA, B...",1,198213,False,https://api.spotify.com/v1/tracks/0I1eFRytp4XR...,0I1eFRytp4XRhLCjT6tZm7,False,I Can't Handle Change,83,...,42vd9Cgr89pjb0Jipvaj5d,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",I Can't Handle Change,2010-03-14,day,6,album,spotify:album:42vd9Cgr89pjb0Jipvaj5d,ushm81020595,https://open.spotify.com/track/0I1eFRytp4XRhLC...
1,[{'external_urls': {'spotify': 'https://open.s...,"[AD, AE, AG, AL, AM, AO, AR, AT, AU, AZ, BA, B...",1,203346,False,https://api.spotify.com/v1/tracks/0ntQJM78wzOL...,0ntQJM78wzOLVeCUAW7Y45,False,Sex on Fire,81,...,5CZR6ljD0x9fTiS4mh9wMp,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",Only By The Night,2008-09-23,day,11,album,spotify:album:5CZR6ljD0x9fTiS4mh9wMp,USRC10800300,https://open.spotify.com/track/0ntQJM78wzOLVeC...
2,[{'external_urls': {'spotify': 'https://open.s...,"[AD, AE, AG, AL, AM, AO, AR, AT, AU, AZ, BA, B...",1,153294,False,https://api.spotify.com/v1/tracks/6N22FZs2ZhPB...,6N22FZs2ZhPBYi3b9XPajV,False,Still Don't Know My Name,80,...,788r22Bhr3IHLTNPy8pZea,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",Euphoria (Original Score from the HBO Series),2019-10-04,day,26,album,spotify:album:788r22Bhr3IHLTNPy8pZea,USQX91902507,https://open.spotify.com/track/6N22FZs2ZhPBYi3...
3,[{'external_urls': {'spotify': 'https://open.s...,"[CA, US]",2,151826,False,https://api.spotify.com/v1/tracks/1Fwj0wThn3kT...,1Fwj0wThn3kTg8D7KgWdsU,False,My Tears Are Becoming A Sea,62,...,6R0ynY7RF20ofs9GJR5TXR,"[{'height': 640, 'url': 'https://i.scdn.co/ima...","Hurry Up, We're Dreaming",2011,year,22,album,spotify:album:6R0ynY7RF20ofs9GJR5TXR,GB55H1100012,https://open.spotify.com/track/1Fwj0wThn3kTg8D...
4,[{'external_urls': {'spotify': 'https://open.s...,"[AD, AE, AG, AL, AM, AO, AR, AT, AU, AZ, BA, B...",1,371413,False,https://api.spotify.com/v1/tracks/7Jh1bpe76CNT...,7Jh1bpe76CNTCgdgAdBw4Z,False,Heroes - 2017 Remaster,75,...,4I5zzKYd2SKDgZ9DRf5LVk,"[{'height': 640, 'url': 'https://i.scdn.co/ima...","""Heroes"" (2017 Remaster)",1977,year,10,album,spotify:album:4I5zzKYd2SKDgZ9DRf5LVk,USJT11700005,https://open.spotify.com/track/7Jh1bpe76CNTCgd...


## Get Audio Features for All Tracks

In [6]:
audio_feat_df = pd.json_normalize(sp.audio_features(tracks=playlist_df['track_id'].tolist()))

# change column header 'id' to 'track_id'
audio_feat_df.rename(columns={'id':'track_id'}, inplace=True)

audio_feat_df.head()

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,track_id,uri,track_href,analysis_url,duration_ms,time_signature
0,0.247,0.438,5,-8.478,1,0.0358,0.044700,0.000867,0.111,0.3900,185.711,audio_features,0I1eFRytp4XRhLCjT6tZm7,spotify:track:0I1eFRytp4XRhLCjT6tZm7,https://api.spotify.com/v1/tracks/0I1eFRytp4XR...,https://api.spotify.com/v1/audio-analysis/0I1e...,198213,3
1,0.542,0.905,9,-5.653,1,0.0540,0.001720,0.010400,0.136,0.3740,153.398,audio_features,0ntQJM78wzOLVeCUAW7Y45,spotify:track:0ntQJM78wzOLVeCUAW7Y45,https://api.spotify.com/v1/tracks/0ntQJM78wzOL...,https://api.spotify.com/v1/audio-analysis/0ntQ...,203347,4
2,0.316,0.630,5,-5.963,1,0.1170,0.471000,0.262000,0.205,0.3130,89.643,audio_features,6N22FZs2ZhPBYi3b9XPajV,spotify:track:6N22FZs2ZhPBYi3b9XPajV,https://api.spotify.com/v1/tracks/6N22FZs2ZhPB...,https://api.spotify.com/v1/audio-analysis/6N22...,153294,1
3,0.194,0.376,4,-11.356,1,0.0463,0.110000,0.386000,0.593,0.0723,48.637,audio_features,1Fwj0wThn3kTg8D7KgWdsU,spotify:track:1Fwj0wThn3kTg8D7KgWdsU,https://api.spotify.com/v1/tracks/1Fwj0wThn3kT...,https://api.spotify.com/v1/audio-analysis/1Fwj...,151827,3
4,0.490,0.758,7,-6.491,1,0.0297,0.000671,0.484000,0.092,0.4350,112.114,audio_features,7Jh1bpe76CNTCgdgAdBw4Z,spotify:track:7Jh1bpe76CNTCgdgAdBw4Z,https://api.spotify.com/v1/tracks/7Jh1bpe76CNT...,https://api.spotify.com/v1/audio-analysis/7Jh1...,371413,4


## Get Artist Data

In [18]:
artist_json = sp.artists(artists=artist_id_ls)
artist_df = pd.json_normalize(artist_json['artists'])

# change column header 'id' to 'track_id'
artist_df.rename(columns={'id':'artist_id'}, inplace=True)

# replace all '.' with '_' in column headers
artist_df.columns = artist_df.columns.str.replace("[.]", "_")

artist_df.head()

,genres,href,artist_id,images,name,popularity,type,uri,external_urls_spotify,followers_href,followers_total
0,"[indie pop, weirdcore]",https://api.spotify.com/v1/artists/7dIpKWlEeAl...,7dIpKWlEeAljA20vFJ82RD,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",Roar,69,artist,spotify:artist:7dIpKWlEeAljA20vFJ82RD,https://open.spotify.com/artist/7dIpKWlEeAljA2...,None,295837
1,"[modern rock, rock]",https://api.spotify.com/v1/artists/2qk9voo8llS...,2qk9voo8llSGYcZ6xrBzKx,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",Kings of Leon,78,artist,spotify:artist:2qk9voo8llSGYcZ6xrBzKx,https://open.spotify.com/artist/2qk9voo8llSGYc...,None,4896158
2,"[indie poptimism, pop]",https://api.spotify.com/v1/artists/2feDdbD5ara...,2feDdbD5araYcm6JhFHHw7,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",Labrinth,81,artist,spotify:artist:2feDdbD5araYcm6JhFHHw7,https://open.spotify.com/artist/2feDdbD5araYcm...,None,1772541
3,"[french shoegaze, french synthpop, indietronic...",https://api.spotify.com/v1/artists/63MQldklfxk...,63MQldklfxkjYDoUE4Tppz,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",M83,73,artist,spotify:artist:63MQldklfxkjYDoUE4Tppz,https://open.spotify.com/artist/63MQldklfxkjYD...,None,1992490
4,"[art rock, classic rock, glam rock, permanent ...",https://api.spotify.com/v1/artists/0oSGxfWSnnO...,0oSGxfWSnnOXhD2fKuz2Gy,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",David Bowie,81,artist,spotify:artist:0oSGxfWSnnOXhD2fKuz2Gy,https://open.spotify.com/artist/0oSGxfWSnnOXhD...,None,7581724


## Clean Data

### 1. Playlist Data Frame

In [19]:
# Celan release date in playlist_df
playlist_df['release_date_clean'] = pd.to_datetime(playlist_df['track_album_release_date'])

# Get release year in playlist_df
playlist_df['release_date_year'] = playlist_df['release_date_clean'].dt.strftime('%Y')

playlist_df.head()

,added_at,is_local,primary_color,added_by_external_urls_spotify,added_by_href,added_by_id,added_by_type,added_by_uri,track_album_album_type,track_album_artists,...,track_popularity,track_preview_url,track_track,track_track_number,track_type,track_uri,video_thumbnail_url,release_date_clean,release_date_year,artist_id
0,2020-11-18T06:14:13Z,False,None,https://open.spotify.com/user/toricopeland_,https://api.spotify.com/v1/users/toricopeland_,toricopeland_,user,spotify:user:toricopeland_,single,[{'external_urls': {'spotify': 'https://open.s...,...,83,https://p.scdn.co/mp3-preview/4c2554c0416d42a4...,True,1,track,spotify:track:0I1eFRytp4XRhLCjT6tZm7,None,2010-03-14,2010,7dIpKWlEeAljA20vFJ82RD
1,2020-11-18T06:13:55Z,False,None,https://open.spotify.com/user/toricopeland_,https://api.spotify.com/v1/users/toricopeland_,toricopeland_,user,spotify:user:toricopeland_,album,[{'external_urls': {'spotify': 'https://open.s...,...,81,https://p.scdn.co/mp3-preview/8cca3506fa42dd9c...,True,3,track,spotify:track:0ntQJM78wzOLVeCUAW7Y45,None,2008-09-23,2008,2qk9voo8llSGYcZ6xrBzKx
2,2020-11-18T06:31:22Z,False,None,https://open.spotify.com/user/toricopeland_,https://api.spotify.com/v1/users/toricopeland_,toricopeland_,user,spotify:user:toricopeland_,album,[{'external_urls': {'spotify': 'https://open.s...,...,80,https://p.scdn.co/mp3-preview/2f3df1d334b61c34...,True,12,track,spotify:track:6N22FZs2ZhPBYi3b9XPajV,None,2019-10-04,2019,2feDdbD5araYcm6JhFHHw7
3,2020-11-18T06:19:50Z,False,None,https://open.spotify.com/user/toricopeland_,https://api.spotify.com/v1/users/toricopeland_,toricopeland_,user,spotify:user:toricopeland_,album,[{'external_urls': {'spotify': 'https://open.s...,...,62,https://p.scdn.co/mp3-preview/b320a486166d237d...,True,1,track,spotify:track:1Fwj0wThn3kTg8D7KgWdsU,None,2011-01-01,2011,63MQldklfxkjYDoUE4Tppz
4,2020-11-18T07:40:50Z,False,None,https://open.spotify.com/user/toricopeland_,https://api.spotify.com/v1/users/toricopeland_,toricopeland_,user,spotify:user:toricopeland_,album,[{'external_urls': {'spotify': 'https://open.s...,...,75,https://p.scdn.co/mp3-preview/92caa5df4a00974b...,True,3,track,spotify:track:7Jh1bpe76CNTCgdgAdBw4Z,None,1977-01-01,1977,0oSGxfWSnnOXhD2fKuz2Gy


## Join Data Frames

In [20]:
df_track_ls = [playlist_df, tracks_df, audio_feat_df]

track_main = playlist_df.merge(tracks_df, on='track_id', how='left')

In [21]:
# merge all track data frames into the playlist data frame
df_track_ls = [playlist_df, tracks_df, audio_feat_df]
track_df_merged = reduce(lambda  left,right: pd.merge(left,right,on=['track_id'],
                                            how='left'), df_track_ls)

# merge artist data frame into the merge data frame
df_main = track_df_merged.merge(artist_df, on='artist_id', how='left')
df_main.head()

,added_at,is_local_x,primary_color,added_by_external_urls_spotify,added_by_href,added_by_id,added_by_type,added_by_uri,track_album_album_type,track_album_artists,...,genres,href_y,images,name_y,popularity_y,type,uri,external_urls_spotify_y,followers_href,followers_total
0,2020-11-18T06:14:13Z,False,None,https://open.spotify.com/user/toricopeland_,https://api.spotify.com/v1/users/toricopeland_,toricopeland_,user,spotify:user:toricopeland_,single,[{'external_urls': {'spotify': 'https://open.s...,...,"[indie pop, weirdcore]",https://api.spotify.com/v1/artists/7dIpKWlEeAl...,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",Roar,69,artist,spotify:artist:7dIpKWlEeAljA20vFJ82RD,https://open.spotify.com/artist/7dIpKWlEeAljA2...,None,295837
1,2020-11-18T06:13:55Z,False,None,https://open.spotify.com/user/toricopeland_,https://api.spotify.com/v1/users/toricopeland_,toricopeland_,user,spotify:user:toricopeland_,album,[{'external_urls': {'spotify': 'https://open.s...,...,"[modern rock, rock]",https://api.spotify.com/v1/artists/2qk9voo8llS...,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",Kings of Leon,78,artist,spotify:artist:2qk9voo8llSGYcZ6xrBzKx,https://open.spotify.com/artist/2qk9voo8llSGYc...,None,4896158
2,2020-11-18T06:31:22Z,False,None,https://open.spotify.com/user/toricopeland_,https://api.spotify.com/v1/users/toricopeland_,toricopeland_,user,spotify:user:toricopeland_,album,[{'external_urls': {'spotify': 'https://open.s...,...,"[indie poptimism, pop]",https://api.spotify.com/v1/artists/2feDdbD5ara...,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",Labrinth,81,artist,spotify:artist:2feDdbD5araYcm6JhFHHw7,https://open.spotify.com/artist/2feDdbD5araYcm...,None,1772541
3,2020-11-18T06:31:22Z,False,None,https://open.spotify.com/user/toricopeland_,https://api.spotify.com/v1/users/toricopeland_,toricopeland_,user,spotify:user:toricopeland_,album,[{'external_urls': {'spotify': 'https://open.s...,...,"[indie poptimism, pop]",https://api.spotify.com/v1/artists/2feDdbD5ara...,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",Labrinth,81,artist,spotify:artist:2feDdbD5araYcm6JhFHHw7,https://open.spotify.com/artist/2feDdbD5araYcm...,None,1772541
4,2020-11-18T06:19:50Z,False,None,https://open.spotify.com/user/toricopeland_,https://api.spotify.com/v1/users/toricopeland_,toricopeland_,user,spotify:user:toricopeland_,album,[{'external_urls': {'spotify': 'https://open.s...,...,"[french shoegaze, french synthpop, indietronic...",https://api.spotify.com/v1/artists/63MQldklfxk...,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",M83,73,artist,spotify:artist:63MQldklfxkjYDoUE4Tppz,https://open.spotify.com/artist/63MQldklfxkjYD...,None,1992490


## Verify Cleaning

In [22]:
df_main.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 66 entries, 0 to 65
Data columns (total 98 columns):
 #   Column                              Non-Null Count  Dtype         
---  ------                              --------------  -----         
 0   added_at                            66 non-null     object        
 1   is_local_x                          66 non-null     bool          
 2   primary_color                       0 non-null      object        
 3   added_by_external_urls_spotify      66 non-null     object        
 4   added_by_href                       66 non-null     object        
 5   added_by_id                         66 non-null     object        
 6   added_by_type                       66 non-null     object        
 7   added_by_uri                        66 non-null     object        
 8   track_album_album_type              66 non-null     object        
 9   track_album_artists                 66 non-null     object        
 10  track_album_available_market

In [23]:
df_main.isnull().sum()

added_at                           0
is_local_x                         0
primary_color                     66
added_by_external_urls_spotify     0
added_by_href                      0
                                  ..
type                               0
uri                                0
external_urls_spotify_y            0
followers_href                    66
followers_total                    0
Length: 98, dtype: int64

#### Change mode to major or minor

In [ ]:
mod_dict = {0 : 'minor',
            1: 'major'}

track_df_main['mode_clean'] = track_df_main['mode'].replace(mod_dict, inplace=True)

In [ ]:
mode_data = track_df_main['mode'].value_counts()

plt.figure(figsize=(10,5))
sns.barplot(x=mode_data.index, y=mode_data.values, alpha=0.8)
plt.title("Track Mode Count")
plt.xlabel("Mode", labelpad=14)
plt.ylabel("Count", labelpad=14)

for index, value in enumerate(mode_data):
    plt.text(index,value, str(value))

plt.show()

In [ ]:
#number of times charted by artist
df_numbercharted = track_df_main.groupby('artist_name').mean().sort_values('artist_popularity', ascending=False)
df_numbercharted = df_numbercharted.reset_index()
df_numbercharted

In [ ]:
px.bar(x='artist_name', y='artist_popularity', data_frame=df_numbercharted.head(7), 
       title="Top 7 Artists with Highest Popularity on Spotify")


In [ ]:
f,ax = plt.subplots(figsize=(14,10))
sns.heatmap(track_df_main.corr(),annot = True,fmt = ".1f",ax = ax)
plt.show()

In [ ]:
dance_check = track_df_main.sort_values('release_date_year', ascending=True)
dance_check

In [ ]:
px.line(x='release_date_year', y='danceability', data_frame=dance_check, title="Danceability over the course of the Year")

### Major or Minor